In [14]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
include("utils.jl")
include("hmm_est.jl")

regime_summary (generic function with 1 method)

In [188]:
#columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
assets = [:USA_Net]
data_df = DataFrame(XLSX.readtable("data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │ ACWI_Net    │
├────────────┼────────────┼──────────────┼─────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │   0.0246253 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │   0.0560778 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │   0.0302388 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │   0.0125024 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │  -0.0201209 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │ -0.00119203 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │     0.01818 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │  -0.0566908 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │   -0.042229 │
│

In [189]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1987, 12,1 )) |> x -> to(x, Date(2022,09,30))

417×1 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2022-09-30
┌────────────┬─────────────┐
│            │ USA_Net     │
├────────────┼─────────────┤
│ 1988-01-31 │   0.0410122 │
│ 1988-02-29 │   0.0401109 │
│ 1988-03-31 │  -0.0349774 │
│ 1988-04-30 │  0.00818061 │
│ 1988-05-31 │  0.00760205 │
│ 1988-06-30 │   0.0445733 │
│ 1988-07-31 │  -0.0040323 │
│ 1988-08-31 │   -0.035096 │
│     ⋮      │      ⋮      │
│ 2022-03-31 │     0.03417 │
│ 2022-04-30 │  -0.0952756 │
│ 2022-05-31 │ -0.00270982 │
│ 2022-06-30 │  -0.0868359 │
│ 2022-07-31 │   0.0888659 │
│ 2022-08-31 │  -0.0404618 │
│ 2022-09-30 │  -0.0977404 │
└────────────┴─────────────┘
            402 rows omitted

In [194]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1987, 12,1 )) |> x -> to(x, Date(2022,09,30))
returns_summarystats(period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1489,1.4788,0.0928,0.4192,-2.257,-0.718,-0.1985,0.0114,1.4606,0.6231,0.1528


In [197]:
hmm = hmm_est(period_returns, 3)
regime_summary(hmm[1], assets, 36)

Means
Standard deviations
Regimes transition matrix


Regime,USA_Net
1,2.660
2,0.489
3,-0.972


Regime,USA_Net
1,0.114
2,0.158
3,0.314


"Correlations in regime 1"

,USA_Net
USA_Net,1.000


"Correlations in regime 2"

,USA_Net
USA_Net,1.000


"Correlations in regime 3"

,USA_Net
USA_Net,1.000


Regime,1,2,3
1,0.128,0.533,0.339
2,0.000,0.937,0.063
3,0.257,0.000,0.743


In [192]:
n_assets = length(assets)
scenarios = simulate_hmm(hmm[1], n_assets, 180, 100_000)

for a in 1:n_assets
    print_percentiles(scenarios[a,:,:], [0.1, 0.3, 0.5, 0.7, 0.9], 12, string(assets[a]))
end 

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.0757,-0.1335,-0.1372,-0.1402,-0.1389,-0.137,-0.1396,-0.1381,-0.136,-0.1381,-0.1359,-0.1376,-0.1394,-0.1373,-0.1361
0.3,0.0617,0.0366,0.0341,0.0335,0.035,0.0357,0.034,0.0349,0.0343,0.0353,0.0353,0.0344,0.0344,0.035,0.0352
0.5,0.1301,0.1185,0.1177,0.1168,0.118,0.1181,0.1176,0.1185,0.1174,0.1179,0.118,0.117,0.118,0.1176,0.1175
0.7,0.1897,0.1872,0.1864,0.1861,0.1869,0.1865,0.1864,0.1869,0.1862,0.1865,0.1866,0.1862,0.1868,0.1866,0.1865
0.9,0.2724,0.2771,0.2767,0.2766,0.277,0.2776,0.2775,0.2773,0.2773,0.2769,0.2775,0.2757,0.278,0.277,0.2771


In [193]:
#weights = [0.62, 0.26, 0.12]
weights = [.7, .3]
acwi_cum_ret = cumprod(scenarios .+ 1, dims=3)

acwi = zeros(100_000, 180)
for t in 1:180
    acwi[:,t] = acwi_cum_ret[:,:,t]' * weights * 100
end

acwi = acwi[:, 2:end] ./ acwi[:,1:end-1] .-1

print_percentiles(expm1.(acwi),[0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95], 12)

DimensionMismatch: DimensionMismatch: second dimension of A, 1, does not match length of x, 2